### Player rating: Self-Comparisoin

In [2]:
import pandas as pd
import seaborn as sns

In [7]:
data = pd.read_csv('test_data/2021_fantasy_scores.csv')
data = data[['player_name','player_id','matchup','game_date','fantasy_score','fantasy_score_cumul']]

In [14]:
fs_range = data[['player_name','fantasy_score']].groupby('player_name').agg({'fantasy_score':['max','min']})
fs_range.columns = ['max_fantasy_score','min_fantasy_score']
fs_range = fs_range.sort_values(by='max_fantasy_score',ascending=False).reset_index()
fs_range

,player_name,max_fantasy_score,min_fantasy_score
0,Nikola Jokic,84.3,21.9
1,Kyrie Irving,83.3,11.3
2,Karl-Anthony Towns,79.0,16.4
3,Giannis Antetokounmpo,78.5,16.0
4,Trae Young,78.1,8.1
...,...,...,...
532,David Johnson,0.0,-2.0
533,Jaime Echenique,0.0,0.0
534,Jordan Goodwin,-0.8,-2.0
535,Matt Ryan,-1.0,-1.0
